In [1]:
import sys
import pandas as pd
import numpy as np
import sklearn
import requests
import re
import time

In [2]:
#format URL, adding appropriate headers
def URL_format(url):
    http_flag = 0
    www_flag = 0
    new_url = ""
    
    if "http" not in url[:4]:
        http_flag = 1
    if "www" not in url[:11]:
        www_flag = 1

    if http_flag == 1:
        try:
            r = requests.get('https://'+ url, timeout=10)
            new_url = "https://"
        except:
            new_url = "http://"
            
        if www_flag == 1:
            new_url = new_url + "www."
            
        new_url = new_url+url
    else:
        if www_flag == 1:
            if url[:5] == "https":
                new_url = url[:8] + "www." + url[8:]
            elif url[:4] == "http":
                new_url = url[:7] + "www." + url[7:]
        else:
            new_url = url
    time.sleep(1/4)
    return new_url

#Length of URL
def URL_len(url):
    return len(url)

#Length of Hostname
def hostname_len(url):
    #assumes we start with ://www....
    matches = re.findall("://www.([\w\-\.]+)", str(url))
    result = 0
    if (len(matches) >= 1):
        result = len(matches[0])
    return result

#Length of path
def path_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url))[0])
    return match_len

#Length of first directory
def first_dir_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url))[0])
    return match_len
    
#Length of top level domain
def top_level_len(url):
    domains = re.findall("://www.([\w\-\.]+)", str(url))
    if len(domains) != 0:
        tld = domains[0].split('.')
        return len(tld[-1])
    return 0

#Count of '-'
def count_dash(url):
    dashes = 0
    for char in url:
        if char == '-':
            dashes += 1     
    return dashes

#Count of '@'
def count_at(url):
    at = 0
    for char in url:
        if char == '@':
            at += 1     
    return at

#Count of '?'
def count_q(url):
    q = 0
    for char in url:
        if char == '?':
            q += 1     
    return q

#Count of '%'
def count_percent(url):
    percent = 0
    for char in url:
        if char == '%':
            percent += 1     
    return percent

#Count of '.'
def count_dot(url):
    dot = 0
    for char in url:
        if char == '.':
            dot += 1     
    return dot

#Count of '='
def count_equal(url):
    eq = 0
    for char in url:
        if char == '=':
            eq += 1     
    return eq

#Count of ';'
def count_colon(url):
    colon = 0
    for char in url:
        if char == ';':
            colon += 1     
    return colon

#Count of 'www'
def count_www(url):
    www = 0
    www = len(re.findall("www", str(url)))
    return www

#Count of numbers
def count_numbers(url):
    numbers = sum(c.isdigit() for c in url)
    return numbers

#Count of letters
def count_letters(url):
    letters = sum(c.isalpha() for c in url)
    return letters

#Count of directories
def dir_count(url):
    dirs = 0
    for char in url:
        if char == '/':
            dirs = dirs + 1
    #assumes that it starts with ://www....
    return dirs - 2

#Count of single letter directories
def single_letter_dir(url):
    sldir = 0
    sldir = len(re.findall("\/[\w]\/", str(url)))
    return sldir
                
#Count of queries
def query_count(url):
    count = 0
    count = len(re.findall("\?\w+(&?\w+)*", str(url)))
    return count

#Ratio of uppercase to lowercase letters
def ratio_upper_lower(url):
    upperSum = 0
    lowerSum = 0
    for char in url:
        if char.isupper():
            upperSum = upperSum + 1
        if char.islower():
            lowerSum = lowerSum + 1
    # if url has 0 of one, this will return 0
    if lowerSum == 0:
        return upperSum
    return upperSum / lowerSum

#IP vs not
def is_ip(url):
    if len(re.findall("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", str(url))) > 0:
        return 1
    else:
        return 0
    
# Shortened or not
def is_shortened(url):
    shortened = { "bit.ly", "tinyurl", "goo.gl", "ow.ly", "t.co", 
                 "tiny.cc", "bit.do", "shorte.st", "cutt.ly", "clkim"}
    
    if any(shrt in url for shrt in shortened):
        return 1
    else:
        return 0

#HTTP vs. HTTPS
#Returns 1 if HTTPS, 0 if HTTP
def is_https(url):
    if url[:5] == 'https':
        return 1
    elif url[:4] == 'http':
        return 0


In [3]:
#Read csv data using pandas
majestic_million=pd.read_csv('majestic_million.csv')
phishtank=pd.read_csv('phistank_verified_online_data.csv')

#Only take the first 10,000 values of the majestic million values
majestic_million=majestic_million.head(10000)

#Rows are store in res[] after being processed
res=[]

malicious=0
benign=1


In [ ]:

for index,row in majestic_million.iterrows():
    #print(row.IDN_Domain)
    url = URL_format(row.IDN_Domain)
    print(index)
    print(url)

    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,benign,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])
  
result_column_names = ['url','malicious','url_res','hostname_res','path_res','first_dir_res','top_level_res','dash_res','at_res','q_res','percent_res','dot_res','equal_res','colon_res','www_res','numbers_res','letters_res','dir_res','single_letter_res','query_res','ratio_upper_lower_res','ip_res','shortened_res','http_res']

result_majestic = pd.DataFrame(res,columns=result_column_names)
result_majestic.to_csv('majestic_out.csv', sep=',',index=False)

0
https://www.google.com
1
https://www.facebook.com
2
https://www.youtube.com
3
https://www.twitter.com
4
https://www.instagram.com
5
https://www.linkedin.com
6
https://www.microsoft.com
7
https://www.apple.com
8
https://www.wikipedia.org
9
https://www.googletagmanager.com
10
http://www.youtu.be
11
https://www.en.wikipedia.org
12
https://www.plus.google.com
13
https://www.pinterest.com
14
https://www.play.google.com
15
https://www.vimeo.com


In [4]:
res=[]
for index,row in phishtank.iterrows():
    #print("original: " + row.url)
    url = row.url
    print(index)
    print(url)
    
    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,malicious,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])
result_column_names = ['url','malicious','url_res','hostname_res','path_res','first_dir_res','top_level_res','dash_res','at_res','q_res','percent_res','dot_res','equal_res','colon_res','www_res','numbers_res','letters_res','dir_res','single_letter_res','query_res','ratio_upper_lower_res','ip_res','shortened_res','http_res']

result = pd.DataFrame(res,columns=result_column_names)
result.to_csv('phistank_out.csv', sep=',',index=False)

0
http://www.mysqleg.secure.runescape.com-gf.ru/
1
https://www.mysqleg.secure.runescape.com-gf.ru/
2
http://www.emaig.secure.runescape.com-gf.ru/
3
http://emaig.secure.runescape.com-gf.ru/
4
http://mysqleg.secure.runescape.com-gf.ru/
5
https://mysqleg.secure.runescape.com-gf.ru/
6
https://www.emaig.secure.runescape.com-gf.ru/
7
http://www.mww.48tees.runescape.com-gf.ru/
8
https://www.mww.48tees.runescape.com-gf.ru/
9
https://emaig.secure.runescape.com-gf.ru/
10
https://mww.48tees.runescape.com-gf.ru/
11
http://mww.48tees.runescape.com-gf.ru/
12
https://olx-pl.safeorder.cyou/1623165790744
13
http://olx.pl-myitem.pw/cash96937365
14
https://olx.pl-myitem.pw/cash96937365
15
http://www.wiitzar.com/cms/MX10/index.html
16
https://rakuten.jp.mnhcrf.shop/
17
http://rakuten.jp.mnhcrf.shop/
18
https://services.runescape.com-vzla.ru/m=weblogin/loginform874,732,675,24156382,2167
19
https://rakuten.jp.snjcrc.shop/
20
http://rakuten.jp.snjcrc.shop/
21
https://elimed.com.ec/wp-includes/i.html#lauraran

http://amazon.stswin520j.top/signim/?openid_pape_max_auth_age=http:/specs.openid.net/auth/2.0/identifier&amp;openid_return_to=https:/www.amazon.co.jp/?ref_=nav_em_hd_re_signin&amp;openid.identity=_select&amp;openid.assoc_handle=jpflex&amp;openid.mode=checkid_setup&amp;key=a@b.c&amp;openid.claimed_id=http:/specs.openid.net/auth/2.0/identifier_select&amp;openid.ns=http:/specs.openid.net/auth/2.0&amp;ref_=nav_em_hd_clc_signin
619
http://cheks2021.000webhostapp.com/
620
http://www.doors.lk/ctct/
621
https://adelfasys.com/config/log/in/82cadb0649a3af4968404c9f6031b233/
622
http://grupwahotnew.duckdns.org/
623
http://lnsteqram.beget.tech/
624
http://lnsteqrams.beget.tech/
625
http://pemersatubangsa-indo.duckdns.org/
626
http://vrifikasionnpaqessfbb.000webhostapp.com/
627
https://www.zononm.co.u2tywg.cn/mobile/
628
http://gruptante-novia.duckdns.org/
629
http://chatwhatsappgroups00044.duckdns.org/
630
http://guddutravelworld.com/ama/7e2ef
631
http://whatisnbphgsbn.duckdns.org/
632
https://msd

http://join-chat551.hotbkpviral56.duckdns.org/
1148
http://heyirv.com/Folder
1149
http://heyirv.com/Folder/
1150
http://www.fastel.ph/TO/authorize_client_id:rh5lje2o-thaw-ub0w-ntsw-mfioy2den1sz_wjv9f0dpxknsa1i5mt62h784cgezlqo3byurpojbv7hlu9tq1d30c8gyf6wnzr5eim4aksx24qw90s2b1jlgv5rehi8ta3cfxn7m6ykuopdz?data=ZGVtb0B3aGF0ZXZlci5sb2w=
1151
http://itaucard-atualizacao-cadastral2.azurewebsites.net/
1152
http://verifications-pages.duckdns.org/
1153
http://www.teknozain.com/TO/authorize_client_id:axgyqwpk-pwzi-bg3j-zgmo-wur5ps8cgoxy_q3kbiepg62vx4dm7z9af8ys1wl50nochujrtqz8uw1gikhmlj9xyvte7352cbnf4apodrs60ijnkxpvlab95egsf0ucy8dzhmtoq341r7w62?data=bWFza2VkQG1hc2tlZC5jb20=
1154
http://594cd78d2116.ngrok.io/instagram
1155
http://a0548407.xsph.ru/postale/authentification/colis=947/
1156
https://fbcom-mdzogliao.homebet88.me/connect.html?timelimit=a459553b10064b9e20a16c47e52291ff
1157
http://prayermeetingplayback.us/files/
1158
http://verifications-pages.duckdns.org/verifications-security-pages.php
11

1665
http://supergaca.com/
1666
https://expensereduction.s3.eu-de.cloud-object-storage.appdomain.cloud/1chenner.html
1667
http://nicenecei.geronmonet.cn/?GHVthXseZzERDXftGVYBHUnjIMjnKBjFCxrdXFTCHyjgJNUHgvHHFCdgJHVJFcdRXDftc
1668
http://configurrations-accntconfrim.ml/Login_support.php
1669
http://nicenecei.geronmonet.cn/?GHVth*seZzERDXftGVYBHUnjIMinKBjFC*rdXFTCHyigJNUHgvHFCdgJHVJFcdR*Dftc
1670
https://galsecus-galsiaonlisnes-ports.000webhostapp.com/
1671
https://validerment.000webhostapp.com/
1672
https://mewwwww.000webhostapp.com/
1673
http://winwithtesla.com/
1674
http://winwithtesla.com
1675
https://ee.review-newbilling.com/
1676
http://dfcc.termcupboard.link/apmix
1677
http://nicenecei.geronmonet.cn/?GHVthXsseZzERDXftGVYBHUnjIMjnKBjFCxrdXFTCHyjgJNUHgvHFCdgJHVJFcdRXDftc
1678
http://www.cutt.ly/onhME0y/
1679
http://gabung-grupbokep2021.duckdns.org/
1680
http://107.172.140.104/
1681
http://107.172.140.104/pc/index.php?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.smbc.

http://www_jacss_co_jp.2rple8.rest/stat/stat.php
2238
https://www_jacss_co_jp.2rple8.rest/stat/stat.php
2239
https://www_jacss_co_jp.2rple8.rest/
2240
http://accountliveout.freecluster.eu/
2241
http://nicenecei.geronmonet.cn/?GHVthX$eZzERDXftGVYBHUni1MjnKBiFC*rd*FTCHyjgINUHgvHFCdgJHVIFcdR*Dftc
2242
http://sig0n04.mipropia.com/index.html?i=1
2243
http://whitelistwallet.com
2244
https://whitelistwallet.com/
2245
https://www_jacss_co_jp.4et6d.rest/
2246
http://amazon.litingzxw.cn/signin/?openid_pape_max_auth_age=http:/specs.openid.net/auth/2.0/identifier&amp;openid_return_to=https:/www.amazon.co.jp/?ref_=nav_em_hd_re_signin&amp;openid.identity=_select&amp;openid.assoc_handle=jpflex&amp;openid.mode=checkid_setup&amp;key=a@b.c&amp;openid.claimed_id=http:/specs.openid.net/auth/2.0/identifier_select&amp;openid.ns=http:/specs.openid.net/auth/2.0&amp;ref_=nav_em_hd_clc_signin
2247
http://accounts-autoscout24.de/login/authentication.html
2248
http://accounts-autoscout24.de/login/twofactorauthent

...

https://machinta.com/
8658
http://paypal.com.service.id999.sorttheweb.com/paypal/a5a006b7e9cd5dd226cbee369112e326MTVjOWQzOTRiMTUzMTk3YWVmNGM0ZTdjNjFiNjEzNDM=
8659
http://paypal.com.service.id999.sorttheweb.com/paypal/a5a006b7e9cd5dd226cbee369112e326MTVjOWQzOTRiMTUzMTk3YWVmNGM0ZTdjNjFiNjEzNDM=/
8660
http://paypal.com.service.id999.sorttheweb.com/paypal/8b19408a1996d326aa823cabbfd0548aNzM1YTlkYmYzNzcyYTY2NzA0NmEyMmYyMDRjNmNjNzM=
8661
http://paypal.com.service.id999.sorttheweb.com/paypal/8b19408a1996d326aa823cabbfd0548aNzM1YTlkYmYzNzcyYTY2NzA0NmEyMmYyMDRjNmNjNzM=/
8662
http://paypal.com.service.id999.sorttheweb.com/paypal/6d19466c2eb82a4abe9cf7cf847308e5MTEwYTcyZjI5NDg0ZTNmY2JlMDZjOTAxMTNkOWU3MzA=/
8663
http://paypal.com.service.id999.sorttheweb.com/paypal/6d19466c2eb82a4abe9cf7cf847308e5MTEwYTcyZjI5NDg0ZTNmY2JlMDZjOTAxMTNkOWU3MzA=
8664
http://paypal.com.service.id999.sorttheweb.com/paypal/4532ea0bb3fc10618de3caad74383a68YTFlMjYxMjQzZmQyMjU1YzhmZWZiNmNhZTUwMWQzYTE=
8665
http://paypal.com.

http://e-leclerc.fr-epicerie.club/
9107
https://betasusturkiye.blogspot.com/
9108
http://173.212.239.242/
9109
https://share.hsforms.com/1OWoQghkBQDO-dfBLtGExeQ4g63f
9110
https://hindmovie.cc/
9111
http://millenniaco-my.sharepoint.com/personal/vgibbs_mhmltd_com/_layouts/15/doc.aspx?sourcedoc={ec69f793-5e80-4ab9-ab2a-66e49de9b392}&amp;action=default&amp;slrid=dca0489f-f03b-b000-9fd0-1e171e182306&amp;originalpath=ahr0chm6ly9tawxszw5uawfjby1tes5zagfyzxbvaw50lmnvbs86bzovzy9wzxjzb25hbc92z2liynnfbwhtbhrkx2nvbs9fcfazywv5qvhybetxexbtnuozchm1suiwaw1pnk90znrqcdrpsnpmqmu0uhlrp3j0aw1lpujzclb1vkrnmtbn&amp;cid=0e1475ff-8901-48a8-aeae-660a9e5b5547
9112
http://keypointtraining-my.sharepoint.com/personal/janeann_keypoint-training_com/_layouts/15/doc.aspx?sourcedoc={9af291d0-87c8-456b-8c74-dddd4a2e5852}&amp;action=default&amp;slrid=5e9d489f-500f-a000-704a-3a9b1d01a72a&amp;originalpath=ahr0chm6ly9rzxlwb2ludhryywluaw5nlw15lnnoyxjlcg9pbnquy29tlzpvoi9nl3blcnnvbmfsl2phbmvhbm5fa2v5cg9pbnqtdhjhaw5pbmdfy29tl0v0

9541
https://onedrive.live.com/view.aspx?resid=357CF596B048E521!68171&amp;ithint=onenote%2c&amp;authkey=!AGRJG_pGeGj6PeQ
9542
http://floorsdirectltd.co.uk/chase/surf4.php
9543
http://floorsdirectltd.co.uk/chase/surf3.php
9544
https://onedrive.live.com/view.aspx?resid=B116B3793040630B!5852&amp;ithint=onenote%2c&amp;authkey=!ALTmjf-4BzB1YGU
9545
https://www.shortlink.net/VERIFY-ATM
9546
https://norcaltc-my.sharepoint.com/personal/acolon_norcaltc_org/_layouts/15/WopiFrame.aspx?guestaccesstoken=YzE2SVUs6Et1YFBzBrRbp0zBlkd6FtBulRUE02WUDHw=&amp;docid=1_1f1c059892dd04acf92bca72fa2b86901&amp;wdFormId={22FA7E1D-2B31-41B8-A33F-0D3A531F6142}&amp;action=formsubmit
9547
https://umzap.com/PROTOCOLO-(197019774904)
9548
https://wu7q5.app.link/?platform=hootsuite
9549
http://inaceinox.pt/js/vendor/security/Update/myaccount/signin
9550
http://inaceinox.pt/image_temp/default/Update/myaccount/signin/
9551
https://norcaltc-my.sharepoint.com/personal/acolon_norcaltc_org/_layouts/15/WopiFrame.aspx?guestacces

9896
http://re-redirection-pp-account-id98763432.blogspot.com/
9897
http://lp2m.umsu.ac.id/wp-includes/widgets/redirect.html
9898
https://re-redirection-acc-id923872635122.blogspot.com/
9899
https://webexert.com
9900
http://encryptdrive.booogle.net/?state={ids:[1zsW2sDoRx1jgl1y1_XDtGIxXpkoMS27F]
9901
https://wilsonvaluation602-my.sharepoint.com/:o:/g/personal/brian_wilsonvaluation_com/EvgZsh2f49NAtJi6I_LnklcBZ46LeDpzwPCKxS6JGi7zMw?e=un6Z
9902
http://www.piandizano.it/sito/wp-content/gallery/css/
9903
http://lp2m.umsu.ac.id/wp-includes/IXR/redirect.html
9904
https://auduboninstitute-my.sharepoint.com/:o:/g/personal/kramsey_auduboninstitute_org/EveSQu6pzsxOjjLjb-YGjEwBwb6DV_wN9eBMuZghm1jKBw?e=bCYSTA
9905
http://web1577.webbox444.server-home.org/E6DF44E1650D6EA3856C4F1B59DACA9A
9906
https://soude-masi.firebaseapp.com/
9907
https://docs.google.com/viewerng/viewer?url=proxy.ge.tt/1/files/8tNpiBy2/0/blob?referrer%3Duser-uR6z6NgiUCtfxJqXnhc2bXYONVSMVcncQdrvc-%26pdf
9908
http://nebojsega.com/k

http://jeffreybcam.net/wp-admin/new/administrator_restore.htm
10264
http://0s.nrxwo2lo.ozvs4y3pnu.cmla.ru/?act=login
10265
http://hetershaven.net/css/03496773043/09572370984380/001/3B762723737B2z.html
10266
http://www.i-m.mx/eupdate/emailaccountupdate/
10267
http://getactive365.com/wp-includes/css/upgrade/web-upgrade/
10268
http://mala-riba.com/view/
10269
http://fpmaam.org/antsafiderana/AMAFI/webmail-email-upgrading-service/MicrosoftEmailUpgrade.html
10270
http://indiankitchenfood.com/menu/status.php
10271
http://souvenirsplace.com/chuakad.php
10272
http://www.ezblox.site/free/jennifer111/helpdesk
10273
http://albel.intnet.mu/File/download_doc.php
10274
http://dongsuh.net/visa/index.html/?/visa.com.br/programa/de/pontos/cadastro.html
10275
http://thescrapescape.com/cccxxxxxxxxxc/doc/doc/processing.html
10276
http://www.thescrapescape.com/cccxxxxxxxxxc/doc/doc/processing.html
10277
http://www.thescrapescape.com/cccxxxxxxxxxc/doc/doc/document.php
10278
http://albel.intnet.mu/File/file_d